In [ ]:
import os
import shutil
import random



In [ ]:
original_dataset_dir ='/content/drive/MyDrive/Colab Notebooks/image_classification/dataset'

In [ ]:
base_dir = '/content/drive/MyDrive/Colab Notebooks/image_classification/dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
for folder in [base_dir,train_dir,test_dir]:
  if not os.path.exists(folder):
    os.makedirs(folder)

In [ ]:
train_split = 0.8
test_split = 0.2

In [ ]:
# Loop through each class
for class_name in os.listdir(original_dataset_dir):
    class_path = os.path.join(original_dataset_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [img for img in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, img))]
    random.shuffle(images)

    train_size = int(len(images) * train_split)
    train_images = images[:train_size]
    test_images = images[train_size:]

    # Create class subfolders
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Copy images
    for img in train_images:
        src_path = os.path.join(class_path, img)
        dst_path = os.path.join(train_class_dir, img)
        if os.path.isfile(src_path):  # Add this check
            shutil.copy(src_path, dst_path)
    for img in test_images:
        src_path = os.path.join(class_path, img)
        dst_path = os.path.join(test_class_dir, img)
        if os.path.isfile(src_path):  # Add this check
            shutil.copy(src_path, dst_path)

print("✅ Dataset split complete!")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (128, 128)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train for validation
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
from tensorflow.keras import layers, models

num_classes = train_data.num_classes

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=img_size+(3,)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {test_acc:.2f}")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/content/drive/MyDrive/Colab Notebooks/image_classification/dataset/test/Bear/Bear_27.jpeg'  # path to any test image
img = image.load_img(img_path, target_size=img_size)
x = image.img_to_array(img) / 255.0
x = np.expand_dims(x, axis=0)

pred = model.predict(x)
class_names = list(train_data.class_indices.keys())
print("Predicted Class:", class_names[np.argmax(pred)])